for breif matrix explanations read-> https://github.com/USEPA/USEEIO_API/blob/master/doc/data_format.md

In [1]:
#BASE URL AND KEY
#Use the requests library
import csv
import zipfile, io
import os
import pathlib
import requests as r
import pandas as pd
pd.options.display.max_rows = 999

base_url ='https://smmtool.app.cloud.gov/api/' 
api_headers = {}
# api-key for USEEIO
with open(".USEEIO_API_KEY", 'r') as KEY_FILE:
    api_headers['x-api-key']=KEY_FILE.read()

In [2]:
# AVAILABLE MODELS
# Show the available models in the native JSON output
models = r.get(base_url+'/models',headers=api_headers)
models_json = models.json()

# MODEL NAME
#Use the first available model
model_name = models_json[0]['id']

#MODEL SECTORS
#Get the sectors
model_sectors = r.get(base_url+model_name+'/sectors',headers=api_headers)
model_sectors_json = model_sectors.json()
model_sector_ids = []
for i in model_sectors_json:
    model_sector_ids.append(i['id'])
first_sector_id = model_sector_ids[0]
first_sector_name = model_sectors_json[0]['name']
print('The '+ model_name +' model has ' + str(len(model_sectors_json)) + ' sectors.')

# INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
HC_indictr_id = model_indictrs_json[8]['id']
model_indictrs_names = []
model_indictrs_ids = []
for i in model_indictrs_json:
    model_indictrs_names.append(i['name'])
    model_indictrs_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_indictrs_json)) + ' indicators. The names are:')
print(model_indictrs_names)

The USEEIO model has 388 sectors.
The USEEIO model has 24 indicators. The names are:
['Acid Rain', 'Energy', 'Freshwater Aquatic Ecotoxicity', 'Eutrophication', 'Food Waste', 'Global Climate Change', 'Hazardous Air Pollutants', 'Hazardous Waste', 'Human Health Cancer', 'Human Health Noncancer', 'Human Health - Respiratory Effects', 'Human Health Cancer and Noncancer', 'Employment', 'Land', 'Metals', 'Minerals and Metals', 'Muncipal Solid Waste', 'Nonrenewable Energy', 'Ozone Depletion', 'Pesticides', 'Renewable Energy', 'Smog Formation', 'Value Added', 'Water']


In [3]:
model_indictrs_ids

['ACID',
 'ENRG',
 'ETOX',
 'EUTR',
 'FOOD',
 'GCC',
 'HAPS',
 'HAZW',
 'HC',
 'HNC',
 'HRSP',
 'HTOX',
 'JOBS',
 'LAND',
 'METL',
 'MINE',
 'MSW',
 'NREN',
 'OZON',
 'PEST',
 'REN',
 'SMOG',
 'VADD',
 'WATR']

In [4]:
#INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
model_indictrs_json

[{'id': 'ACID',
  'index': 0,
  'name': 'Acid Rain',
  'code': 'ACID',
  'unit': 'kg SO2 eq',
  'group': 'Impact Potential'},
 {'id': 'ENRG',
  'index': 1,
  'name': 'Energy',
  'code': 'ENRG',
  'unit': 'MJ',
  'group': 'Resource Use'},
 {'id': 'ETOX',
  'index': 2,
  'name': 'Freshwater Aquatic Ecotoxicity',
  'code': 'ETOX',
  'unit': 'CTUe',
  'group': 'Impact Potential'},
 {'id': 'EUTR',
  'index': 3,
  'name': 'Eutrophication',
  'code': 'EUTR',
  'unit': 'kg N eq',
  'group': 'Impact Potential'},
 {'id': 'FOOD',
  'index': 4,
  'name': 'Food Waste',
  'code': 'FOOD',
  'unit': 'kg',
  'group': 'Waste Generated'},
 {'id': 'GCC',
  'index': 5,
  'name': 'Global Climate Change',
  'code': 'GCC',
  'unit': 'kg CO2 eq',
  'group': 'Impact Potential'},
 {'id': 'HAPS',
  'index': 6,
  'name': 'Hazardous Air Pollutants',
  'code': 'HAPS',
  'unit': 'kg',
  'group': 'Chemical Releases'},
 {'id': 'HAZW',
  'index': 7,
  'name': 'Hazardous Waste',
  'code': 'HAZW',
  'unit': 'kg',
  'group

In [5]:
# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
# D0 = D0_response.json()
# sorted(D0)

D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()

# sorted(D0[0])

In [6]:
#
# Collect ALL Indicators from ALL Industr Sectors and Limit to Georgia
#

# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()

indicator_data = {}
# Add the industry sector IDs
# indicator_data.update({'industry':model_sector_ids})
indicator_data.update({'industry_code':[_.split('/')[0] for _ in model_sector_ids]})
indicator_data.update({'industry_detail':[_.split('/')[1] for _ in model_sector_ids]})
indicator_data.update({'industry_region':[_.split('/')[2] for _ in model_sector_ids]})


for key, values_list in zip(model_indictrs_ids, D0):
    indicator_data.update({key:values_list})

all_indic_all_indust = pd.DataFrame(data=indicator_data)
# all_indic_all_indust.loc[0:20,['industry_code', 'industry_detail', 'ACID']].sort_values(by='ACID', ascending=False)

all_indic_all_indust.head()

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us,0.003038,0.000000,31.608836,0.011122,0.000026,0.558816,0.000025,...,3.697394e-09,0.0,0.000099,0.0,0.000000e+00,3.661750e-04,0.000000,0.022389,0.471064,0.002513
1,1111b0,"fresh wheat, corn, rice, and other grains",us,0.020069,0.000000,12.615068,0.032458,0.000122,1.978520,0.000067,...,1.154869e-06,0.0,0.000472,0.0,2.278307e-09,6.212053e-04,0.000000,0.055404,-0.002548,0.460768
2,111200,"fresh vegetables, melons, and potatoes",us,0.007174,0.003225,28.396543,0.006305,0.001194,0.457168,0.000285,...,2.445284e-06,0.0,0.004637,0.0,7.327268e-06,1.075716e-03,0.003225,0.021458,0.604597,0.591335
3,111300,fresh fruits and tree nuts,us,0.003656,0.000000,20.718815,0.000275,0.001945,0.286005,0.000038,...,1.547424e-05,0.0,0.007557,0.0,1.289248e-06,4.691275e-04,0.000000,0.018263,0.605345,0.456172
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us,0.000952,0.000005,0.000677,0.000118,0.001786,0.709750,0.000044,...,2.509153e-08,0.0,0.006971,0.0,8.190052e-10,3.562148e-14,0.000005,0.037800,0.534020,0.046371


---
### Load USEEIO <--> NAICS 2012 Concordance

In [8]:
# Load USEEIO <--> NAICS Concordance
data_dir = pathlib.Path.cwd().parent / 'data_raw'
useeio_keys = pd.read_csv(data_dir / 'USEEIO-NAICS-Translation' / 'USEEIO-NAICS-Concordance_2012.csv', header=5).loc[0:958,:]


def expand_keys(key_string):
    
    keys = []
    
    for key_subset in key_string.split(','):

        key_subset = key_subset.strip()
        try:
            start = key_subset.split('-')[0]
            end = key_subset.split('-')[1]

            try:
                key = int(start)
                keys.append(key)

                while str(key)[-1] != end:
                    key += 1
                    keys.append(key)
                    
            except ValueError:
                keys.append('*')
                
        except IndexError:
            try:
                keys.append(int(key_subset))
            except ValueError:
                keys.append('*')
                
    return keys


useeio_naic_concordance = useeio_keys[['Detail', 'Related 2012 NAICS Codes']].dropna().reset_index(drop=True)
useeio_naic_concordance['Detail'] = useeio_naic_concordance['Detail'].apply(lambda x: str.lower(x))

useeio_naic_concordance = useeio_naic_concordance.rename(columns={'Detail': 'industry_code'})

useeio_naic_concordance['relavant_naics_codes'] = useeio_naic_concordance.apply(lambda x: expand_keys(x["Related 2012 NAICS Codes"]), axis=1)
useeio_naic_concordance.head()

,industry_code,Related 2012 NAICS Codes,relavant_naics_codes
0,1111a0,11111-2,"[11111, 11112]"
1,1111b0,"11113-6, 11119","[11113, 11114, 11115, 11116, 11119]"
2,111200,1112,[1112]
3,111300,1113,[1113]
4,111400,1114,[1114]


In [9]:
all_indic_all_indust = all_indic_all_indust[
    all_indic_all_indust['industry_code'].isin(useeio_naic_concordance['industry_code'].tolist())
]

In [10]:
all_indic_all_indust.head()

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us,0.003038,0.000000,31.608836,0.011122,0.000026,0.558816,0.000025,...,3.697394e-09,0.0,0.000099,0.0,0.000000e+00,3.661750e-04,0.000000,0.022389,0.471064,0.002513
1,1111b0,"fresh wheat, corn, rice, and other grains",us,0.020069,0.000000,12.615068,0.032458,0.000122,1.978520,0.000067,...,1.154869e-06,0.0,0.000472,0.0,2.278307e-09,6.212053e-04,0.000000,0.055404,-0.002548,0.460768
2,111200,"fresh vegetables, melons, and potatoes",us,0.007174,0.003225,28.396543,0.006305,0.001194,0.457168,0.000285,...,2.445284e-06,0.0,0.004637,0.0,7.327268e-06,1.075716e-03,0.003225,0.021458,0.604597,0.591335
3,111300,fresh fruits and tree nuts,us,0.003656,0.000000,20.718815,0.000275,0.001945,0.286005,0.000038,...,1.547424e-05,0.0,0.007557,0.0,1.289248e-06,4.691275e-04,0.000000,0.018263,0.605345,0.456172
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us,0.000952,0.000005,0.000677,0.000118,0.001786,0.709750,0.000044,...,2.509153e-08,0.0,0.006971,0.0,8.190052e-10,3.562148e-14,0.000005,0.037800,0.534020,0.046371


In [11]:
all_indic_all_indust = (all_indic_all_indust
 .set_index('industry_code')
 .join(useeio_naic_concordance
      .set_index('industry_code')
      )
)

all_indic_all_indust.to_csv(data_dir / 'USEEIO_Industry_Indicators' / 'indicators.csv')

In [12]:
all_indic_all_indust.head()

,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,HAZW,...,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR,Related 2012 NAICS Codes,relavant_naics_codes
industry_code,,,,,,,,,,,,,,,,,,,,,
1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us,0.003038,0.000000,31.608836,0.011122,0.000026,0.558816,0.000025,5.724197e-08,...,0.000099,0.0,0.000000e+00,3.661750e-04,0.000000,0.022389,0.471064,0.002513,11111-2,"[11111, 11112]"
1111b0,"fresh wheat, corn, rice, and other grains",us,0.020069,0.000000,12.615068,0.032458,0.000122,1.978520,0.000067,2.694699e-07,...,0.000472,0.0,2.278307e-09,6.212053e-04,0.000000,0.055404,-0.002548,0.460768,"11113-6, 11119","[11113, 11114, 11115, 11116, 11119]"
111200,"fresh vegetables, melons, and potatoes",us,0.007174,0.003225,28.396543,0.006305,0.001194,0.457168,0.000285,2.645463e-06,...,0.004637,0.0,7.327268e-06,1.075716e-03,0.003225,0.021458,0.604597,0.591335,1112,[1112]
111300,fresh fruits and tree nuts,us,0.003656,0.000000,20.718815,0.000275,0.001945,0.286005,0.000038,4.604881e-06,...,0.007557,0.0,1.289248e-06,4.691275e-04,0.000000,0.018263,0.605345,0.456172,1113,[1113]
111400,"greenhouse crops, mushrooms, nurseries, and fl...",us,0.000952,0.000005,0.000677,0.000118,0.001786,0.709750,0.000044,4.400720e-06,...,0.006971,0.0,8.190052e-10,3.562148e-14,0.000005,0.037800,0.534020,0.046371,1114,[1114]
